In [1]:
import numpy as np
import random
from random import randint
from gurobipy import *
import pandas as pd
from random import seed

In [2]:
Manufacturing_plants = 2
Distribution = 3
Market = 4
Products = 2
Outsourced = 2
epsilon = 25

In [152]:
# Scenario parameters
a_si = [[1,1], [1,0], [0,1]] # don't include [0,0]
b_sj = [[1,1,1], [1,0,1], [1,1,0], [1,0,0], [0,1,1], [0,1,0], [0,0,1]] # don't include [0,0,0]

In [153]:
Scenarios = [[x,y] for x in a_si for y in b_sj]

In [154]:
Scenarios

[[[1, 1], [1, 1, 1]],
 [[1, 1], [1, 0, 1]],
 [[1, 1], [1, 1, 0]],
 [[1, 1], [1, 0, 0]],
 [[1, 1], [0, 1, 1]],
 [[1, 1], [0, 1, 0]],
 [[1, 1], [0, 0, 1]],
 [[1, 0], [1, 1, 1]],
 [[1, 0], [1, 0, 1]],
 [[1, 0], [1, 1, 0]],
 [[1, 0], [1, 0, 0]],
 [[1, 0], [0, 1, 1]],
 [[1, 0], [0, 1, 0]],
 [[1, 0], [0, 0, 1]],
 [[0, 1], [1, 1, 1]],
 [[0, 1], [1, 0, 1]],
 [[0, 1], [1, 1, 0]],
 [[0, 1], [1, 0, 0]],
 [[0, 1], [0, 1, 1]],
 [[0, 1], [0, 1, 0]],
 [[0, 1], [0, 0, 1]]]

In [155]:
num_Scenarios = len(Scenarios)
p_scen = 1/num_Scenarios

In [156]:
# Product Demand
np.random.seed(0)
demand = np.random.randint(0,50,(num_Scenarios, Products,Market))

In [157]:
# Cost of opening
f_i = [200, 50]
f_j = [75, 100, 50]

In [158]:
# Unit cost of manufacturing product 
np.random.seed(0)
Manufacturing_costs = np.random.uniform(0,2, (Manufacturing_plants,Products))

In [159]:
Manufacturing_costs

array([[1.09762701, 1.43037873],
       [1.20552675, 1.08976637]])

In [160]:
# Unit cost of transporting m from plant to DC
np.random.seed(0)
Transportation_i_j = np.random.uniform(0,2, (Products, Manufacturing_plants, Distribution))

In [161]:
# Unit cost of transporting m from DC to Market Zone
np.random.seed(0)
Transportation_j_k = np.random.uniform(0,2, (Products, Distribution, Market))

In [162]:
# Plant Capacities: Bigger capacities for the more expensive ones
np.random.seed(0)

Capacities_i = np.zeros(Manufacturing_plants) # in volume (metres cubed)
Capacities_i[0] = np.random.randint(800,1000)
Capacities_i[1] = np.random.randint(200,400) 
Capacities_j = np.zeros(Distribution) # in volume (metres cubed)
Capacities_j[0] = np.random.randint(400, 600)
Capacities_j[1] = np.random.randint(600, 800)
Capacities_j[2] = np.random.randint(200,400)
Capacities_l = np.random.randint(50,100, (Products,Outsourced)) # in terms of products 


In [163]:
# Cost of purchasing product m from supplier l (assume only 1 product type from each outsourcer)
np.random.seed(0)

levels = 2
Supplier_cost = np.zeros((levels, Products, Outsourced))
Supplier_cost[0] = np.random.uniform(10,15, (Products, Outsourced))
Supplier_cost[1] = np.random.randint(15,20, (Products, Outsourced))

In [164]:
Supplier_cost

array([[[12.74406752, 13.57594683],
        [13.01381688, 12.72441591]],

       [[16.        , 18.        ],
        [17.        , 19.        ]]])

In [165]:
# Cost of transporting product m from outsourced facility l to j
np.random.seed(0)

T_O_DC = np.random.uniform(2, 5, (Products, Outsourced, Distribution))

In [166]:
# Cost of shipping product m from outsourced facility l to k
np.random.seed(0)

T_O_MZ = np.random.uniform(5, 7,(Products, Outsourced, Market))

In [167]:
# Product volume 
np.random.seed(0)

volume = np.random.uniform(2,3,(Products))

In [168]:
# unit cost of lost sales 
np.random.seed(0)

lost_sales = np.random.randint(18,25,(Market,Products))

In [169]:
lost_sales

array([[22, 23],
       [18, 21],
       [21, 21],
       [19, 21]])

In [170]:
# Initialize model variables
x_i = {} # opening manufacturing plant
x_j = {} # opening DC
U_km = {} # quantity lost sales
V_lm = {} # quantity products purchased from outsourcing
Q_im = {} # quantity produced
Y_ijm = {} # shipping i -> j
Z_jkm = {} # shipping j -> k
T_ljm = {} # shipping l -> j
T_lkm = {} # shipping l -> k
y_lm = {} # indicator variable for step function 

# Dictionaries for analysis 
Cost_dict = {}
Summary_dict = {} 

In [171]:
## Model

In [172]:
grbModel = Model('synthetic')

In [173]:
def SetGurobiModel():
    
    for i in range(Manufacturing_plants):
        x_i[i] = grbModel.addVar(vtype = GRB.BINARY)
    
    for j in range(Distribution):
        x_j[j] = grbModel.addVar(vtype = GRB.BINARY)
    
    for s in range(num_Scenarios):
        for k in range(Market):
            for m in range(Products):
                U_km[s,k,m] = grbModel.addVar(vtype = GRB.INTEGER)
        
    for s in range(num_Scenarios):
        for m in range(Products):
            for l in range(Outsourced):
                V_lm[s,m,l] = grbModel.addVar(vtype = GRB.INTEGER)
    
    for s in range(num_Scenarios):
        for m in range(Products):
            for i in range(Manufacturing_plants):
                Q_im[s,m,i] = grbModel.addVar(vtype = GRB.INTEGER)
            
    for s in range(num_Scenarios):  
        for m in range(Products):
            for i in range(Manufacturing_plants):
                for j in range(Distribution):
                    Y_ijm[s,m,i,j] = grbModel.addVar(vtype = GRB.INTEGER)
                
    
    for s in range(num_Scenarios):
        for m in range(Products):
            for j in range(Distribution):
                for k in range(Market): 
                    Z_jkm[s,m,j,k] = grbModel.addVar(vtype = GRB.INTEGER)                
    
    for s in range(num_Scenarios):
        for m in range(Products):
            for l in range(Outsourced):
                for j in range(Distribution):
                    T_ljm[s,m,l,j] = grbModel.addVar(vtype = GRB.INTEGER)      
    
    for s in range(num_Scenarios):
        for m in range(Products):
            for l in range(Outsourced):
                for k in range(Market):
                    T_lkm[s,m,l,k] = grbModel.addVar(vtype = GRB.INTEGER)
                
    
    for s in range(num_Scenarios):
        for m in range(Products):
            for l in range(Outsourced):
                y_lm[s,m,l] = grbModel.addVar(vtype = GRB.BINARY)             
       
                
    SetGrb_Obj()
    ModelCons()

In [174]:
def SolveModel():
    
    grbModel.params.OutputFlag = 0
    grbModel.optimize()
    
    # get variable values 
    v_val_x_i = grbModel.getAttr('x', x_i)
    v_val_x_j = grbModel.getAttr('x', x_j)
    v_val_U_km = grbModel.getAttr('x', U_km)
    v_val_V_lm = grbModel.getAttr('x', V_lm)
    v_val_Q_im = grbModel.getAttr('x', Q_im)
    v_val_Y_ijm = grbModel.getAttr('x', Y_ijm)
    v_val_Z_jkm = grbModel.getAttr('x', Z_jkm)
    v_val_T_ljm = grbModel.getAttr('x', T_ljm)
    v_val_T_lkm = grbModel.getAttr('x', T_lkm)
    v_val_y_lm = grbModel.getAttr('x', y_lm)
    
    obj = grbModel.getObjective()
    print("obj val: ", obj.getValue())   
    
    Summary_dict['ObjVal'] = grbModel.objval
    Summary_dict["OpenMPs"] = np.sum(v_val_x_i.values())
    Summary_dict["OpenDCs"] = np.sum(v_val_x_j.values())
    
    for s in range(num_Scenarios):        
        Summary_dict["Purchasing_" + str(s)] = np.round(sum([v_val_V_lm[(s,m,l)] for m in range(Products) for l in range(Outsourced)]))   
        Summary_dict["Production_" + str(s)] = np.round(sum([v_val_Q_im[(s,m,i)] for m in range(Products) for i in range(Manufacturing_plants)]))
        Summary_dict["LostSales_" + str(s)] = np.round(sum([v_val_U_km[(s,k,m)] for m in range(Products) for k in range(Market)]))
        Summary_dict["OutsourceToDC_" + str(s)] = np.round(sum([v_val_T_ljm[(s,m,l,j)] for m in range(Products) for l in range(Outsourced) for j in range(Distribution)]))
        Summary_dict["OutsourceToMarket_" + str(s)] = np.round(sum([v_val_T_lkm[(s,m,l,k)] for m in range(Products) for l in range(Outsourced) for k in range(Market)])) 
    
    Cost_dict["Opening"] =  get_opening_costs(v_val_x_i, v_val_x_j)
    
    for s in range(num_Scenarios):
        Cost_dict["InHouseShipping_" + str(s)] = get_shipping_costs(s,v_val_Y_ijm, v_val_Z_jkm, v_val_T_ljm, v_val_T_lkm)[0]
        Cost_dict["OutsourceShipping_" + str(s)] = get_shipping_costs(s,v_val_Y_ijm, v_val_Z_jkm, v_val_T_ljm, v_val_T_lkm)[1]
        Cost_dict["Production_" + str(s)] = get_production_cost(s,v_val_Q_im)
        Cost_dict["Purchasing_" + str(s)] = get_purchase_costs(s,v_val_V_lm, v_val_y_lm)
        Cost_dict["LostSales_" + str(s)] = get_lost_cost(s,v_val_U_km)    
    
    return

In [175]:
# Objective

def SetGrb_Obj():

    grb_expr = LinExpr()

    # Cost of opening
    OC_1 = 0
    OC_2 = 0
    for i in range(Manufacturing_plants):
        OC_1 += f_i[i]*x_i[i]
    for j in range(Distribution):
        OC_2 += f_j[j]*x_j[j]    
    
    total_shipment = 0    
    total_pr_cost = 0
    total_b_cost = 0
    total_l_cost = 0
    
    # Shipment 
        
    for s in range(num_Scenarios):
        ship_1 = 0
        ship_2 = 0
        ship_3 = 0
        ship_4 = 0
        for i in range(Manufacturing_plants):
            for j in range(Distribution):
                for m in range(Products):
                    ship_1 += Transportation_i_j[m][i][j]*Y_ijm[s,m,i,j]

        for j in range(Distribution):
            for k in range(Market):
                for m in range(Products):
                    ship_2 += Transportation_j_k[m][j][k]*Z_jkm[s,m,j,k]

        for l in range(Outsourced):
            for j in range(Distribution):
                for m in range(Products):
                    ship_3 += T_O_DC[m][l][j]*T_ljm[s,m,l,j]

        for l in range(Outsourced):
            for k in range(Market):
                for m in range(Products):
                    ship_4 += T_O_MZ[m][l][k]*T_lkm[s,m,l,k]
                    
        total_shipment += ship_1 + ship_2 + ship_3 + ship_4

        # Production
        pr_cost = 0
        for i in range(Manufacturing_plants):
            for m in range(Products):
                pr_cost += Manufacturing_costs[i][m]*Q_im[s,m,i]
                
        total_pr_cost += pr_cost

        # Buying from outsource cost
        b_cost = 0
        for l in range(Outsourced):
            for m in range(Products):
                b_cost += Supplier_cost[0][m][l]*V_lm[s,m,l]*(1 - y_lm[s,m,l]) + (Supplier_cost[0][m][l]*epsilon + Supplier_cost[1][m][l]*(V_lm[s,m,l] - epsilon))*y_lm[s,m,l]
                
        total_b_cost += b_cost

        #Lost Sales
        l_cost = 0
        for k in range(Market):
            for m in range(Products):
                l_cost += lost_sales[k][m]*U_km[s,k,m]
                
        total_l_cost += l_cost

    grb_expr += OC_1 + OC_2 + p_scen*(total_shipment + total_pr_cost + total_b_cost + total_l_cost)
    
    grbModel.setObjective(grb_expr, GRB.MINIMIZE)
    
    return 


In [176]:
# Model Constraints

def ModelCons():
    
    # Network Flow

    grbModel.addConstrs(Q_im[s,m,i] >= quicksum(Y_ijm[s,m,i,j] for j in range(Distribution)) 
                         for s in range(num_Scenarios) for i in range(Manufacturing_plants) for m in range(Products))

    grbModel.addConstrs((quicksum(Y_ijm[s,m,i,j] for i in range(Manufacturing_plants)) +
                         quicksum(T_ljm[s,m,l,j] for l in range(Outsourced))) >= quicksum(Z_jkm[s,m,j,k] for k in range(Market))                        
                        for s in range(num_Scenarios) for j in range(Distribution) for m in range(Products))

    grbModel.addConstrs((quicksum(Z_jkm[s,m,j,k] for j in range(Distribution)) +
                         quicksum(T_lkm[s,m,l,k] for l in range(Outsourced)) + U_km[s,k,m]) >= demand[s][m][k]
                         for s in range(num_Scenarios) for k in range(Market) for m in range(Products)) 
        
                    
    # Purchasing Constraints (everything purchased from outsourced facilities must be shipped)
    grbModel.addConstrs(V_lm[s,m,l] >= quicksum(T_ljm[s,m,l,j] for j in range(Distribution)) + 
                        quicksum(T_lkm[s,m,l,k] for k in range(Market)) for s in range(num_Scenarios) 
                        for m in range(Products) for l in range(Outsourced))
    
    
    # Capacity Constraints
    grbModel.addConstrs(quicksum(volume[m]*Q_im[s,m,i] for m in range(Products)) <= Scenarios[s][0][i]*Capacities_i[i]*x_i[i] 
                        for s in range(num_Scenarios) for i in range(Manufacturing_plants))
    
    grbModel.addConstrs(quicksum(volume[m]*Y_ijm[s,m,i,j] for i in range(Manufacturing_plants) for m in range(Products)) +
                        quicksum(volume[m]*T_ljm[s,m,l,j] for l in range(Outsourced) for m in range(Products)) <= 
                        Scenarios[s][1][j]*Capacities_j[j]*x_j[j] for s in range(num_Scenarios) for s in range(num_Scenarios)
                        for j in range(Distribution))
    
    grbModel.addConstrs((V_lm[s,m,l] <= (Capacities_l[m][l])) for s in range(num_Scenarios)
                        for l in range(Outsourced) for m in range(Products))
    
    
    # Indicator variable constraints for step function (25 is arbitrary)
    grbModel.addConstrs(V_lm[s,m,l] >= (epsilon + 1)*y_lm[s,m,l] for s in range(num_Scenarios) for m in range(Products) for l in range(Outsourced))
    grbModel.addConstrs((V_lm[s,m,l] - (Capacities_l[m][l] - epsilon)*y_lm[s,m,l])
                         <= epsilon for s in range(num_Scenarios) for m in range(Products) for l in range(Outsourced))   
    
    
    return   


In [177]:
def get_opening_costs(x1, x2):
    
    # Cost of opening
    OC_1 = 0
    OC_2 = 0
    for i in range(Manufacturing_plants):
        OC_1 += f_i[i]*x1[i]
    for j in range(Distribution):
        OC_2 += f_j[j]*x2[j]

    Opening = np.round(OC_1 + OC_2)
    
    return(Opening)
   
def get_shipping_costs(scen, Y, Z, T1, T2):
    ship_1 = 0
    ship_2 = 0
    ship_3 = 0
    ship_4 = 0

    # Shipment
    for i in range(Manufacturing_plants):
        for j in range(Distribution):
            for m in range(Products):
                ship_1 += Transportation_i_j[m][i][j]*Y[scen, m,i,j]

    for j in range(Distribution):
        for k in range(Market):
            for m in range(Products):
                ship_2 += Transportation_j_k[m][j][k]*Z[scen,m,j,k]

    for l in range(Outsourced):
        for j in range(Distribution):
            for m in range(Products):
                ship_3 += T_O_DC[m][l][j]*T1[scen,m,l,j]

    for l in range(Outsourced):
        for k in range(Market):
            for m in range(Products):
                ship_4 += T_O_MZ[m][l][k]*T2[scen,m,l,k]
    
    in_house_shipping = np.round(ship_1 + ship_2)

    outsourced_shipping = np.round(ship_3 + ship_4)
    
    return(in_house_shipping, outsourced_shipping)

def get_production_cost(scen, Q):

    # Production
    pr_cost = 0
    for i in range(Manufacturing_plants):
        for m in range(Products):
            pr_cost += Manufacturing_costs[i][m]*Q[scen,m,i]
            
    return(np.round(pr_cost))

def get_purchase_costs(scen, V, y):    

    # Buying from outsource cost
    b_cost = 0
    for l in range(Outsourced):
        for m in range(Products):
            b_cost += Supplier_cost[0][m][l]*V[scen,m,l]*(1 - y[scen,m,l]) + (Supplier_cost[0][m][l]*epsilon + Supplier_cost[1][m][l]*(V[scen,m,l] - epsilon))*y[scen,m,l]            
    return(np.round(b_cost))

def get_lost_cost(scen,U):
    
    #Lost Sales
    l_cost = 0
    for k in range(Market):
        for m in range(Products):
            l_cost += lost_sales[k][m]*U[scen,k,m]
            
    return(np.round(l_cost))

In [178]:
def run_Model():    
        
    SetGurobiModel()
    SolveModel()

In [179]:
def dict_to_dataframe(Dict):
    return pd.DataFrame([list(Dict.values())], columns = list(Dict.keys()))

In [180]:
run_Model()

obj val:  1741.8130506581404


In [181]:
num_correct = 0
for s in range(num_Scenarios):
    if (Summary_dict['Purchasing_'+ str(s)] + Summary_dict['Production_' + str(s) ] + Summary_dict['LostSales_' + str(s)] == 
          np.sum(demand[s])):
        num_correct += 1
    else:
        print(s)
num_correct == num_Scenarios

True

In [182]:
Purchasing = [Summary_dict['Purchasing_' + str(s)] for s in range(num_Scenarios)]
Production = [Summary_dict['Production_' + str(s)] for s in range(num_Scenarios)]
LostSales = [Summary_dict['LostSales_' + str(s)] for s in range(num_Scenarios)]
OutsourceToDC = [Summary_dict['OutsourceToDC_' + str(s)] for s in range(num_Scenarios)]
OutsourceToMarket = [Summary_dict['OutsourceToMarket_' + str(s)] for s in range(num_Scenarios)]

In [183]:
Unit_df = pd.DataFrame(list(zip(Purchasing, Production, LostSales, OutsourceToDC, OutsourceToMarket)), 
             columns = ["Purchasing", "Production", "LostSales", "OutsourceToDC", "OutsourceToMarket"])

In [184]:
Unit_df

,Purchasing,Production,LostSales,OutsourceToDC,OutsourceToMarket
0,0.0,164.0,0.0,0.0,0.0
1,0.0,147.0,0.0,0.0,0.0
2,0.0,249.0,0.0,0.0,0.0
3,0.0,133.0,0.0,0.0,0.0
4,0.0,193.0,0.0,0.0,0.0
5,0.0,168.0,0.0,0.0,0.0
6,64.0,103.0,0.0,0.0,64.0
7,0.0,202.0,0.0,0.0,0.0
8,0.0,197.0,0.0,0.0,0.0
9,0.0,194.0,0.0,0.0,0.0


In [185]:
1 - np.array(list(Unit_df['LostSales']))/(np.array(list(Unit_df['Purchasing'])) + np.array(list(Unit_df['Production'])) + (np.array(list(Unit_df['LostSales']))))

array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 0.78540773, 0.83842795,
       1.        , 0.95918367, 0.80193237, 0.98780488, 0.7826087 ,
       0.80084746])

In [186]:
Purchasing_cost = [Cost_dict['Purchasing_' + str(s)] for s in range(num_Scenarios)]
Production_cost = [Cost_dict['Production_' + str(s)] for s in range(num_Scenarios)]
LostSales_cost = [Cost_dict['LostSales_' + str(s)] for s in range(num_Scenarios)]
InHouseShipping = [Cost_dict['InHouseShipping_' + str(s)] for s in range(num_Scenarios)]
OutsourceShipping = [Cost_dict['OutsourceShipping_' + str(s)] for s in range(num_Scenarios)]

In [187]:
Cost_dict['Opening'] + p_scen*(sum(Purchasing_cost) + sum(Production_cost) + 
                               sum(InHouseShipping) + sum(OutsourceShipping) + sum(LostSales_cost))

1741.7619047619046

In [188]:
stochastic_sol = grbModel.objVal
stochastic_sol

1741.813050658142

In [146]:
Cost_df = pd.DataFrame(list(zip(Purchasing_cost, Production_cost, LostSales_cost, InHouseShipping, OutsourceShipping)), 
             columns = ["Purchasing", "Production", "LostSales", "InHouseShipping", "OutsourceShipping"])

In [147]:
Cost_df

,Purchasing,Production,LostSales,InHouseShipping,OutsourceShipping
0,0.0,182.0,0.0,318.0,0.0
1,0.0,161.0,0.0,304.0,0.0
2,0.0,291.0,0.0,462.0,0.0
3,0.0,145.0,0.0,215.0,0.0
4,0.0,220.0,0.0,424.0,0.0
5,0.0,188.0,0.0,456.0,0.0
6,0.0,110.0,1245.0,288.0,0.0
7,0.0,256.0,0.0,339.0,0.0
8,0.0,258.0,0.0,386.0,0.0
9,0.0,247.0,0.0,386.0,0.0


In [148]:
Summary_dict["OpenMPs"]

2.0

In [149]:
Summary_dict["OpenDCs"]

3.0

## VSS based on nominal scenario

In [43]:
# Initialize model variables
x_i = {} # opening manufacturing plant
x_j = {} # opening DC
U_km = {} # quantity lost sales
V_lm = {} # quantity products purchased from outsourcing
Q_im = {} # quantity produced
Y_ijm = {} # shipping i -> j
Z_jkm = {} # shipping j -> k
T_ljm = {} # shipping l -> j
T_lkm = {} # shipping l -> k
y_lm = {} # indicator variable for step function 

In [44]:
nominal_vals = {}

In [45]:
grbModel_forVSS = Model('VSS')

In [46]:
def SetGurobiModel_forVSS(scen, val):
    
    for i in range(Manufacturing_plants):
        x_i[i] = grbModel_forVSS.addVar(vtype = GRB.BINARY)
    
    for j in range(Distribution):
        x_j[j] = grbModel_forVSS.addVar(vtype = GRB.BINARY)
    
    for k in range(Market):
        for m in range(Products):
            U_km[k,m] = grbModel_forVSS.addVar(vtype = GRB.INTEGER)
        
    for s in range(num_Scenarios):
        for m in range(Products):
            for l in range(Outsourced):
                V_lm[m,l] = grbModel_forVSS.addVar(vtype = GRB.INTEGER)
    
    for m in range(Products):
        for i in range(Manufacturing_plants):
            Q_im[m,i] = grbModel_forVSS.addVar(vtype = GRB.INTEGER)
            
    for m in range(Products):
        for i in range(Manufacturing_plants):
            for j in range(Distribution):
                Y_ijm[m,i,j] = grbModel_forVSS.addVar(vtype = GRB.INTEGER)                
    
    for m in range(Products):
        for j in range(Distribution):
            for k in range(Market): 
                Z_jkm[m,j,k] = grbModel_forVSS.addVar(vtype = GRB.INTEGER)                
    
    for m in range(Products):
        for l in range(Outsourced):
            for j in range(Distribution):
                T_ljm[m,l,j] = grbModel_forVSS.addVar(vtype = GRB.INTEGER)      
    
    for m in range(Products):
        for l in range(Outsourced):
            for k in range(Market):
                    T_lkm[m,l,k] = grbModel_forVSS.addVar(vtype = GRB.INTEGER)                
    
    for m in range(Products):
        for l in range(Outsourced):
            y_lm[m,l] = grbModel_forVSS.addVar(vtype = GRB.BINARY)
                
    SetGrb_Obj_forVSS()
    ModelCons_forVSS(scen, val)

In [47]:
def SolveModel_forVSS(nominal):
    
    grbModel_forVSS.params.OutputFlag = 0
    grbModel_forVSS.optimize()
    v_val_x_i = grbModel_forVSS.getAttr('x', x_i)
    v_val_x_j = grbModel_forVSS.getAttr('x', x_j)
    print(grbModel_forVSS.objVal)
    if nominal == True:        
        nominal_vals['x_i'] = v_val_x_i
        nominal_vals['x_j'] = v_val_x_j    
    return

In [48]:
# Objective

def SetGrb_Obj_forVSS():

    grb_expr = LinExpr()

    # Cost of opening
    OC_1 = 0
    OC_2 = 0
    for i in range(Manufacturing_plants):
        OC_1 += f_i[i]*x_i[i]
    for j in range(Distribution):
        OC_2 += f_j[j]*x_j[j]    
    
    total_shipment = 0    
    total_pr_cost = 0
    total_b_cost = 0
    total_l_cost = 0
    
    # Shipment 
        
    ship_1 = 0
    ship_2 = 0
    ship_3 = 0
    ship_4 = 0
    for i in range(Manufacturing_plants):
        for j in range(Distribution):
            for m in range(Products):
                ship_1 += Transportation_i_j[m][i][j]*Y_ijm[m,i,j]

    for j in range(Distribution):
        for k in range(Market):
            for m in range(Products):
                ship_2 += Transportation_j_k[m][j][k]*Z_jkm[m,j,k]

    for l in range(Outsourced):
        for j in range(Distribution):
            for m in range(Products):
                ship_3 += T_O_DC[m][l][j]*T_ljm[m,l,j]

    for l in range(Outsourced):
        for k in range(Market):
            for m in range(Products):
                ship_4 += T_O_MZ[m][l][k]*T_lkm[m,l,k]
                    
    total_shipment += ship_1 + ship_2 + ship_3 + ship_4

    # Production
    pr_cost = 0
    for i in range(Manufacturing_plants):
        for m in range(Products):
            pr_cost += Manufacturing_costs[i][m]*Q_im[m,i]
                
    total_pr_cost += pr_cost

    # Buying from outsource cost
    b_cost = 0
    for l in range(Outsourced):
        for m in range(Products):
            b_cost += Supplier_cost[0][m][l]*V_lm[m,l]*(1 - y_lm[m,l]) + (Supplier_cost[0][m][l]*epsilon + Supplier_cost[1][m][l]*(V_lm[m,l] - epsilon))*y_lm[m,l]
                
    total_b_cost += b_cost

    #Lost Sales
    l_cost = 0
    for k in range(Market):
        for m in range(Products):
            l_cost += lost_sales[k][m]*U_km[k,m]
                
    total_l_cost += l_cost

    grb_expr += OC_1 + OC_2 + (total_shipment + total_pr_cost + total_b_cost + total_l_cost)
    
    grbModel_forVSS.setObjective(grb_expr, GRB.MINIMIZE)
    
    return 

In [49]:
def ModelCons_forVSS(scen, val):
    
    # Network Flow

    grbModel_forVSS.addConstrs(Q_im[m,i] >= quicksum(Y_ijm[m,i,j] for j in range(Distribution)) 
                         for i in range(Manufacturing_plants) for m in range(Products))
    
    grbModel_forVSS.addConstrs((quicksum(Y_ijm[m,i,j] for i in range(Manufacturing_plants)) +
                         quicksum(T_ljm[m,l,j] for l in range(Outsourced))) >= quicksum(Z_jkm[m,j,k] for k in range(Market))                        
                        for j in range(Distribution) for m in range(Products))
    
    grbModel_forVSS.addConstrs((quicksum(Z_jkm[m,j,k] for j in range(Distribution)) +
                         quicksum(T_lkm[m,l,k] for l in range(Outsourced)) + U_km[k,m]) >= demand[scen][m][k]
                         for k in range(Market) for m in range(Products))
                     
    # Purchasing Constraints (everything purchased from outsourced facilities must be shipped)
    grbModel_forVSS.addConstrs(V_lm[m,l] >= quicksum(T_ljm[m,l,j] for j in range(Distribution)) + 
                        quicksum(T_lkm[m,l,k] for k in range(Market))  
                        for m in range(Products) for l in range(Outsourced))    
    
    # Capacity Constraints
    grbModel_forVSS.addConstrs(quicksum(volume[m]*Q_im[m,i] for m in range(Products)) <= Scenarios[scen][0][i]*Capacities_i[i]*x_i[i] 
                         for i in range(Manufacturing_plants))
    
    grbModel_forVSS.addConstrs(quicksum(volume[m]*Y_ijm[m,i,j] for i in range(Manufacturing_plants) for m in range(Products)) +
                        quicksum(volume[m]*T_ljm[m,l,j] for l in range(Outsourced) for m in range(Products)) <= 
                        Scenarios[scen][1][j]*Capacities_j[j]*x_j[j]  
                        for j in range(Distribution))
    
    grbModel_forVSS.addConstrs((V_lm[m,l] <= (Capacities_l[m][l])) 
                        for l in range(Outsourced) for m in range(Products))
    
    
    # Indicator variable constraints for step function (25 is arbitrary)
    grbModel_forVSS.addConstrs(V_lm[m,l] >= (epsilon + 1)*y_lm[m,l] for m in range(Products) for l in range(Outsourced))
    grbModel_forVSS.addConstrs((V_lm[m,l] - (Capacities_l[m][l] - epsilon)*y_lm[m,l]) <= epsilon 
                               for m in range(Products) for l in range(Outsourced))  
    
    
    # Fixing x variables
    if val:
        grbModel_forVSS.addConstrs(x_i[i] == nominal_vals['x_i'][i] for i in range(Manufacturing_plants))
        grbModel_forVSS.addConstrs(x_j[j] == nominal_vals['x_j'][j] for j in range(Distribution))
    
    return   

In [50]:
def run_Model_forVSS(nominal,scen,val):    
        
    SetGurobiModel_forVSS(scen,val)
    SolveModel_forVSS(nominal)

In [51]:
run_Model_forVSS(1,0,0)

841.7296538177949


In [52]:
Scenario_obj_vals = []

In [53]:
for s in range(num_Scenarios):
    run_Model_forVSS(0,s,1)
    Scenario_obj_vals += [grbModel_forVSS.objval]

841.7296538177949
786.8140044282125
1838.0009303719276
672.2354899819419
4123.02781454227
3673.7431706570164
3629.1983032564094
983.2049159500567
962.9745287071124
910.7477112437641
660.2870943440241
4319.619470491092
4688.686342072732
5061.652035869069
4798.14851458577
3125.811619217513
4018.5955932380184
4275.1495729054
3614.640584116093
4048.1604711439318
5136.256444963406


In [54]:
average_cost = np.average(Scenario_obj_vals)

In [55]:
VSS = average_cost - stochastic_sol

In [56]:
VSS

1217.2611609416874

In [57]:
nominal_vals

{'x_i': {0: 1.0, 1: 0.0}, 'x_j': {0: 1.0, 1: 0.0, 2: 0.0}}

## Expected VSS

In [58]:
# Initialize model variables
x_i = {} # opening manufacturing plant
x_j = {} # opening DC
U_km = {} # quantity lost sales
V_lm = {} # quantity products purchased from outsourcing
Q_im = {} # quantity produced
Y_ijm = {} # shipping i -> j
Z_jkm = {} # shipping j -> k
T_ljm = {} # shipping l -> j
T_lkm = {} # shipping l -> k
y_lm = {} # indicator variable for step function

In [59]:
first_stage_decisions = {}

In [60]:
grbModel_forEVSS = Model('EVSS')

In [61]:
def SetGurobiModel_forEVSS(s1, s2, val):
    
    for i in range(Manufacturing_plants):
        x_i[i] = grbModel_forEVSS.addVar(vtype = GRB.BINARY)
    
    for j in range(Distribution):
        x_j[j] = grbModel_forEVSS.addVar(vtype = GRB.BINARY)
    
    for k in range(Market):
        for m in range(Products):
            U_km[k,m] = grbModel_forEVSS.addVar(vtype = GRB.INTEGER)
        
    for s in range(num_Scenarios):
        for m in range(Products):
            for l in range(Outsourced):
                V_lm[m,l] = grbModel_forEVSS.addVar(vtype = GRB.INTEGER)
    
    for m in range(Products):
        for i in range(Manufacturing_plants):
            Q_im[m,i] = grbModel_forEVSS.addVar(vtype = GRB.INTEGER)
            
    for m in range(Products):
        for i in range(Manufacturing_plants):
            for j in range(Distribution):
                Y_ijm[m,i,j] = grbModel_forEVSS.addVar(vtype = GRB.INTEGER)                
    
    for m in range(Products):
        for j in range(Distribution):
            for k in range(Market): 
                Z_jkm[m,j,k] = grbModel_forEVSS.addVar(vtype = GRB.INTEGER)                
    
    for m in range(Products):
        for l in range(Outsourced):
            for j in range(Distribution):
                T_ljm[m,l,j] = grbModel_forEVSS.addVar(vtype = GRB.INTEGER)      
    
    for m in range(Products):
        for l in range(Outsourced):
            for k in range(Market):
                    T_lkm[m,l,k] = grbModel_forEVSS.addVar(vtype = GRB.INTEGER)                
    
    for m in range(Products):
        for l in range(Outsourced):
            y_lm[m,l] = grbModel_forEVSS.addVar(vtype = GRB.BINARY)
                
    SetGrb_Obj_forEVSS()
    ModelCons_forEVSS(s1, s2, val)


In [133]:
def SolveModel_forEVSS(s1, s2, val):
    grbModel_forEVSS.params.OutputFlag = 0
    grbModel_forEVSS.optimize()
    v_val_x_i = grbModel_forEVSS.getAttr('x', x_i)
    v_val_x_j = grbModel_forEVSS.getAttr('x', x_j)
    if val:
        first_stage_decisions[str(s2) + "_" + "x_i"] = v_val_x_i
        first_stage_decisions[str(s2) + "_" + "x_j"] = v_val_x_j
    return


In [134]:
# Objective

def SetGrb_Obj_forEVSS():

    grb_expr = LinExpr()

    # Cost of opening
    OC_1 = 0
    OC_2 = 0
    for i in range(Manufacturing_plants):
        OC_1 += f_i[i]*x_i[i]
    for j in range(Distribution):
        OC_2 += f_j[j]*x_j[j]    
    
    total_shipment = 0    
    total_pr_cost = 0
    total_b_cost = 0
    total_l_cost = 0
    
    # Shipment 
        
    ship_1 = 0
    ship_2 = 0
    ship_3 = 0
    ship_4 = 0
    for i in range(Manufacturing_plants):
        for j in range(Distribution):
            for m in range(Products):
                ship_1 += Transportation_i_j[m][i][j]*Y_ijm[m,i,j]

    for j in range(Distribution):
        for k in range(Market):
            for m in range(Products):
                ship_2 += Transportation_j_k[m][j][k]*Z_jkm[m,j,k]

    for l in range(Outsourced):
        for j in range(Distribution):
            for m in range(Products):
                ship_3 += T_O_DC[m][l][j]*T_ljm[m,l,j]

    for l in range(Outsourced):
        for k in range(Market):
            for m in range(Products):
                ship_4 += T_O_MZ[m][l][k]*T_lkm[m,l,k]
                    
    total_shipment += ship_1 + ship_2 + ship_3 + ship_4

    # Production
    pr_cost = 0
    for i in range(Manufacturing_plants):
        for m in range(Products):
            pr_cost += Manufacturing_costs[i][m]*Q_im[m,i]
                
    total_pr_cost += pr_cost

    # Buying from outsource cost
    b_cost = 0
    for l in range(Outsourced):
        for m in range(Products):
            b_cost += Supplier_cost[0][m][l]*V_lm[m,l]*(1 - y_lm[m,l]) + (Supplier_cost[0][m][l]*epsilon + Supplier_cost[1][m][l]*(V_lm[m,l] - epsilon))*y_lm[m,l]
                
    total_b_cost += b_cost

    #Lost Sales
    l_cost = 0
    for k in range(Market):
        for m in range(Products):
            l_cost += lost_sales[k][m]*U_km[k,m]
                
    total_l_cost += l_cost

    grb_expr += OC_1 + OC_2 + (total_shipment + total_pr_cost + total_b_cost + total_l_cost)
    
    grbModel_forEVSS.setObjective(grb_expr, GRB.MINIMIZE)
    
    return


In [135]:
def ModelCons_forEVSS(s1, s2, val):
    
    # Network Flow

    grbModel_forEVSS.addConstrs(Q_im[m,i] >= quicksum(Y_ijm[m,i,j] for j in range(Distribution)) 
                         for i in range(Manufacturing_plants) for m in range(Products))

    grbModel_forEVSS.addConstrs((quicksum(Y_ijm[m,i,j] for i in range(Manufacturing_plants)) +
                         quicksum(T_ljm[m,l,j] for l in range(Outsourced))) >= quicksum(Z_jkm[m,j,k] for k in range(Market))                        
                        for j in range(Distribution) for m in range(Products))

    grbModel_forEVSS.addConstrs((quicksum(Z_jkm[m,j,k] for j in range(Distribution)) +
                         quicksum(T_lkm[m,l,k] for l in range(Outsourced)) + U_km[k,m]) >= demand[s1][m][k]
                         for k in range(Market) for m in range(Products))    
        
                    
    # Purchasing Constraints (everything purchased from outsourced facilities must be shipped)
    grbModel_forEVSS.addConstrs(V_lm[m,l] >= quicksum(T_ljm[m,l,j] for j in range(Distribution)) + 
                        quicksum(T_lkm[m,l,k] for k in range(Market))  
                        for m in range(Products) for l in range(Outsourced))    
    
    # Capacity Constraints
    grbModel_forEVSS.addConstrs(quicksum(volume[m]*Q_im[m,i] for m in range(Products)) <= Scenarios[s1][0][i]*Capacities_i[i]*x_i[i] 
                         for i in range(Manufacturing_plants))
    
    grbModel_forEVSS.addConstrs(quicksum(volume[m]*Y_ijm[m,i,j] for i in range(Manufacturing_plants) for m in range(Products)) +
                        quicksum(volume[m]*T_ljm[m,l,j] for l in range(Outsourced) for m in range(Products)) <= 
                        Scenarios[s1][1][j]*Capacities_j[j]*x_j[j]  
                        for j in range(Distribution))
    
    grbModel_forEVSS.addConstrs((V_lm[m,l] <= (Capacities_l[m][l])) 
                        for l in range(Outsourced) for m in range(Products))
    
    
    # Indicator variable constraints for step function (25 is arbitrary)
    grbModel_forEVSS.addConstrs(V_lm[m,l] >= (epsilon + 1)*y_lm[m,l] for m in range(Products) for l in range(Outsourced))
    grbModel_forEVSS.addConstrs((V_lm[m,l] - (Capacities_l[m][l] - epsilon)*y_lm[m,l]) <= epsilon 
                               for m in range(Products) for l in range(Outsourced))
    
    #Fixing first stage decision variables
    if val == 0:            
        grbModel_forEVSS.addConstrs(x_i[i] == first_stage_decisions[str(s2) + "_" + "x_i"][i] for i in range(Manufacturing_plants))
        grbModel_forEVSS.addConstrs(x_j[j] == first_stage_decisions[str(s2) + "_" + "x_j"][j] for j in range(Distribution))
    
    return   


In [136]:
def run_Model_forEVSS(s1, s2, val):    
        
    SetGurobiModel_forEVSS(s1, s2, val)
    SolveModel_forEVSS(s1, s2, val)

In [137]:
# Get dictionary of first stage solutions
for s in range(num_Scenarios):
    run_Model_forEVSS(s,s,1)

In [138]:
v_Det = []

In [139]:
for s in range(num_Scenarios):
    v_det_s = 0
    for j in range(num_Scenarios):
        run_Model_forEVSS(j,s,0)
        v_det_s += grbModel_forEVSS.objval
    v_Det += [v_det_s/num_Scenarios]

In [140]:
average_v_det = np.average(v_Det)

In [141]:
EVSS = average_v_det - stochastic_sol

In [142]:
EVSS

1227.9751626601312

In [153]:
lost_sales

array([[22, 23],
       [18, 21],
       [21, 21],
       [19, 21]])

In [106]:
U_km

{(0, 0, 0): <gurobi.Var C5 (value 0.0)>,
 (0, 0, 1): <gurobi.Var C6 (value 0.0)>,
 (0, 1, 0): <gurobi.Var C7 (value 0.0)>,
 (0, 1, 1): <gurobi.Var C8 (value 0.0)>,
 (0, 2, 0): <gurobi.Var C9 (value -0.0)>,
 (0, 2, 1): <gurobi.Var C10 (value 0.0)>,
 (0, 3, 0): <gurobi.Var C11 (value 0.0)>,
 (0, 3, 1): <gurobi.Var C12 (value 0.0)>,
 (1, 0, 0): <gurobi.Var C13 (value 0.0)>,
 (1, 0, 1): <gurobi.Var C14 (value 0.0)>,
 (1, 1, 0): <gurobi.Var C15 (value 0.0)>,
 (1, 1, 1): <gurobi.Var C16 (value 0.0)>,
 (1, 2, 0): <gurobi.Var C17 (value 0.0)>,
 (1, 2, 1): <gurobi.Var C18 (value 0.0)>,
 (1, 3, 0): <gurobi.Var C19 (value 0.0)>,
 (1, 3, 1): <gurobi.Var C20 (value 0.0)>,
 (2, 0, 0): <gurobi.Var C21 (value 0.0)>,
 (2, 0, 1): <gurobi.Var C22 (value 0.0)>,
 (2, 1, 0): <gurobi.Var C23 (value 0.0)>,
 (2, 1, 1): <gurobi.Var C24 (value 0.0)>,
 (2, 2, 0): <gurobi.Var C25 (value 0.0)>,
 (2, 2, 1): <gurobi.Var C26 (value 0.0)>,
 (2, 3, 0): <gurobi.Var C27 (value 0.0)>,
 (2, 3, 1): <gurobi.Var C28 (value 0.0